## Setup

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

C:\Users\doree\AppData\Local\Temp\ipykernel_3296\2359398814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
data_train = pd.read_csv("../data/BikeRentalDaily_train.csv", sep=";", index_col=0).sort_index()

In [3]:
data_test = pd.read_csv("../data/prepared_test.csv", sep=";", index_col=0).sort_index()

## 1 Minimal Preprocessing

**Create a baseline linear regression model using the originally provided training 
dataset with minimal preprocessing and evaluate it with your validation dataset based on accuracies (MAE) and coefficient of determination (𝑅𝑅2)**

NaN Werte droppen als einfachste Methode, um mit fehlenden Werten umzugehen? Als Preprocessing notwendig, da weiteres Vorgehen nur ohne NaN Werte möglich ist
Imputation Strategien später als Alternative zu Verbesserung der Ergebnisse?

In [4]:
data_train = data_train.dropna()

## 2 Baseline Linear Regression Model

In [5]:
y_label = 'cnt'

features = data_train.drop(columns=[y_label, 'dteday']).columns

X = data_train[features]

y = data_train[y_label]

Erneuter Split des Trainingsdatensets, um ein Validation Datset zu generieren

In [6]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y, 
                                                    test_size=0.2,
                                                    random_state=42)

In [7]:
reg = LinearRegression()

In [ ]:
reg.fit(X, y)

In [ ]:
y_pred = reg.predict(X_validation)

In [ ]:
r2 = r2_score(y_validation, y_pred).round(2)
mae = mean_absolute_error(y_validation, y_pred).round(2)

print(f'R2 Score: {r2}')
print(f'Mean Absolute Error: {mae}')

## 3 Other Options for Missing Value Handling

**Preprocess the original datasets to address the identified data quality issues e.g. Missing values, Outliers, Features to be transformed (e.g. normalization). Check the effect of each preprocessing step with your validation data set* building additional linear regression models**

Einfluss auf R2 und MAE prüfen

Hier jetzt zum Beispiel verschiedene Imputation Strategien

## 4 Outlier Handling

Einfluss auf R2 und MAE prüfen

## 5 Feature Transformation

## 6 Application on Test Dataset

**Finally, check your best model and final data preparation, applying them to the test dataset**

Nur das beste Model wird am Ende auf dem Test Dataset getestet!

## 7 Visualization of Evaluation Results

Export the pre-processed training, validation and test dataset to a CSV